![logo](./img/LogoLine_horizon_C3S.png)

## Tutorial on fetching CARRA data from CDS

CARRA data can be found in the CDS (Copernicus Climate Data Store, https://cds.climate.copernicus.eu/cdsapp#!/home).
You need to create an account and accept the terms and conditions for all datasets that you want to use.


### Fetching CARRA data
The source code will download the month of June 2023. By specifying the start and end date below you can easily download longer periods. We are selecting 2m temperature and single levels. 
Here, we download the 3-hourly analysis time steps and the data will be stored in a specified file.
ERA5 data will also be downloaded, for later comparison with CARRA in the next tutorials.
For more details on ERA5 data download please see the [CDS tutorials in this link](https://github.com/ecmwf-projects/copernicus-training-c3s)


<style>
td, th {
   border: 1px solid white;
   border-collapse: collapse;
}
</style>
<table align="left">
  <tr>
    <th>Run the tutorial via free cloud platforms: </th>
    <th><a href="https://mybinder.org/v2/gh/carlos9917/carra-training-c3s/main?labpath=CARRA_get_T2m.ipynb">
        <img src = "https://mybinder.org/badge.svg" alt = "Binder"></th>
    <th><a href="https://kaggle.com/kernels/welcome?src=https://github.com/carlos9917/carra-training-c3s/blob/main/CARRA_get_T2m.ipynb">
        <img src = "https://kaggle.com/static/images/open-in-kaggle.svg" alt = "Kaggle"></th>
    <th><a href="https://colab.research.google.com/github/carlos9917/carra-training-c3s/blob/main/CARRA_get_T2m.ipynb">
        <img src = "https://colab.research.google.com/assets/colab-badge.svg" alt = "Colab"></th>
  </tr>
</table>

In [6]:
# Here one specifies the domain, year and period to download

yearStart = 2024 #first year to download
yearEnd   = 2024 #last year to download
monthStart =  3 # first month to download
monthEnd   =  3 #last month to download
days = [str(i).zfill(2) for i in range(1,31)] # the list of days to download. Adapt day number for other months
domain = 'west_domain' # the domain to select. Use "east_domain" for the East domain
# the local directory where the data will be stored
DATADIR="CARRA"
#Below a polygon to limit the area downloaded from ERA5, otherwise it will download the whole world
area_for_era5 = [90, -110, 55, 40] #west domain polygon, format: North,West,South,East corners
#area_for_era5 = [85,0,60,50]: east domain polygon
        

---

In [7]:
import cdsapi
import os
# create directory to store CARRA data if it does not exist already:
if not os.path.isdir(DATADIR):
    os.makedirs(os.path.join(DATADIR,"Raw_data"))
c = cdsapi.Client()

# define a helper function to download the data
# Note that the time is hardcoded since we are selecting
# all possible available times. 
def fetch_one_month(month, year, days, domain):
    c.retrieve(
        'reanalysis-carra-single-levels',
        {
            'format': 'grib',
            'domain': domain ,
            'level_type': 'surface_or_atmosphere',
            'variable': '2m_temperature',
            'product_type': 'analysis',
            'year': year,
            'month': month,
            'day': days,
            'time': [
                '00:00', '03:00', '06:00',
                '09:00', '12:00', '15:00',
                '18:00', '21:00',
            ],
        },
        os.path.join(DATADIR,"Raw_data",'T2m_an_'+str(year)+"{:02d}".format(month)+'.grb'))

    



In [8]:
# Loop through the period selected
for year in list(range(yearStart, yearEnd + 1)):
    for month in list(range(1,13)):
        if year == yearStart and month < monthStart:
            # Before the start date.
            continue
        if year == yearEnd and month > monthEnd:
            # After end date.
            break

        print ("Data will be fetched for ", year, month)
        print("This will take a few minutes, depending on the data size (~621 MB for west, ~357 MB for east case)")
        fetch_one_month(month, year, days, domain)

Data will be fetched for  2024 3
This will take a few minutes, depending on the data size (~621 MB for west, ~357 MB for east case)


2024-06-05 19:43:25,937 INFO Welcome to the CDS
2024-06-05 19:43:25,938 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-carra-single-levels
2024-06-05 19:43:26,116 INFO Request is queued
2024-06-05 19:43:58,795 INFO Request is running
2024-06-05 19:44:41,695 INFO Request is completed
2024-06-05 19:44:41,696 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data5/adaptor.mars.external-1717609430.079638-3752-5-89b29582-770c-406f-8a2d-8e6ec9f7e879.grib to CARRA/Raw_data/T2m_an_202403.grb (621.2M)
2024-06-05 19:47:47,972 INFO Download rate 3.3M/s                                                                                            


### Fetching ERA5 data

Here, you downlaod one month of ERA5 data (June 2023). Same as for CARRA, we download 3-hourly data though ERA5 does offer hourly analyses.

---

In [9]:
DATADIR=os.path.join("CARRA","ERA5") 
# create the directory where the data will be stored if it does not exist already

if not os.path.isdir(DATADIR):
    os.makedirs(DATADIR)
    
# Help function to download ERA5 data
def fetch_month_era5(month, year, days, area):
    c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'variable': '2m_temperature',
        'year': str(year),
        'month': str(month).zfill(2),
        'day': days,
        'time': [
            '00:00', '03:00', '06:00',
            '09:00', '12:00', '15:00',
            '18:00', '21:00',
        ],
        'area': area,
        'format': 'netcdf',
    },
     os.path.join(DATADIR,'era5_t2m_'+str(year)+"{:02d}".format(month)+'.grb'))
    
# Loop through the period selected
for year in list(range(yearStart, yearEnd + 1)):
    for month in list(range(1,13)):
        if year == yearStart and month < monthStart:
            # Before the start date.
            continue
        if year == yearEnd and month > monthEnd:
            # After end date.
            break

        print ("ERA5 data will be fetched for ", year, month)
        print("This will take a few minutes, depending on the data size (~39 MB for west area)")
        fetch_month_era5(month, year, days, area_for_era5)

2024-06-05 19:48:13,668 INFO Welcome to the CDS
2024-06-05 19:48:13,669 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-06-05 19:48:13,787 INFO Downloading https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data8/adaptor.mars.internal-1717608329.8119001-29397-4-04898c6c-c581-4d76-b847-9c47bcf39051.nc to CARRA/ERA5/era5_t2m_202403.grb (38.8M)


ERA5 data will be fetched for  2024 3
This will take a few minutes, depending on the data size (~39 MB for west area)


2024-06-05 19:48:25,849 INFO Download rate 3.2M/s                                                                                            
